In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf 
import keras
print(tf.__version__)
print(keras.__version__)
print(pd.__version__)
print(np.__version__)

2.6.0
2.6.0
1.3.4
1.19.5


In [2]:
# custom imports 
import glob
from nn_globals import *
from nn_plotting import gaus, fit_gaus, corr_plot

In [3]:
def _handle_nan_in_x(x):
    x[np.isnan(x)] = 0.0
    x[x==-999.0] = 0.0
    return x

def _zero_out_x(x):
    x = 0.0
    return x
    
def _fixME1Ring(x):
    for i in range(len(x)):
        if (x[i,0] != 0.0): x[i,18] = x[i,18] + 1
    return x   

def muon_data(filename, reg_pt_scale=1.0, reg_dxy_scale=1.0, correct_for_eta=False):
    try:
        logger.info('Loading muon data from {0} ...'.format(filename))
        loaded = np.load(filename)
        the_variables = loaded['variables']
        the_parameters = loaded['parameters']
        # print(the_variables.shape)
        the_variables = the_variables[:nentries]
        the_parameters = the_parameters[:nentries]
        logger.info('Loaded the variables with shape {0}'.format(the_variables.shape))
        logger.info('Loaded the parameters with shape {0}'.format(the_parameters.shape))
    except:
        logger.error('Failed to load data from file: {0}'.format(filename))

    assert(the_variables.shape[0] == the_parameters.shape[0])
    _handle_nan_in_x(the_variables)
      #_fixME1Ring(the_variables)
    _handle_nan_in_x(the_parameters)
    mask = np.logical_or(np.logical_or( np.logical_or((the_variables[:,23] == 11), (the_variables[:,23] == 13)), (the_variables[:,23] == 14)),(the_variables[:,23] == 15)) 

    the_variables = the_variables[mask]  
    the_parameters = the_parameters[mask]  
    assert(the_variables.shape[0] == the_parameters.shape[0])

    x = the_variables[:,0:23]
    y = reg_pt_scale*the_parameters[:,0]
#    print (x[0:30,:], the_variables[0:30,23])
#    print (y[0:30])
    phi = the_parameters[:,1] 
    eta = the_parameters[:,2] 
    vx = the_parameters[:,3] 
    vy = the_parameters[:,4] 
    vz = the_parameters[:,5]      
    dxy = vy * np.cos(phi) - vx * np.sin(phi) 
    logger.info('Loaded the encoded variables with shape {0}'.format(x.shape))
    logger.info('Loaded the encoded parameters with shape {0}'.format(y.shape))

    return x, y, dxy

def muon_data_split(filename, reg_pt_scale=1.0, reg_dxy_scale=1.0, test_size=0.5, correct_for_eta=False):
    x, y, dxy = muon_data(filename, reg_pt_scale=reg_pt_scale, reg_dxy_scale=reg_dxy_scale, correct_for_eta=correct_for_eta)

    # Split dataset in training and testing
    x_train, x_test, y_train, y_test, dxy_train, dxy_test = train_test_split(x, y, dxy, test_size=test_size)
    logger.info('Loaded # of training and testing events: {0}'.format((x_train.shape[0], x_test.shape[0])))

    # Check for cases where the number of events in the last batch could be too few
    validation_split = 0.1
    train_num_samples = int(x_train.shape[0] * (1.0-validation_split))
    val_num_samples = x_train.shape[0] - train_num_samples
    batch_size = 128
    if (train_num_samples%batch_size) < 100:
        logger.warning('The last batch for training could be too few! ({0}%{1})={2}. Please change test_size.'.format(train_num_samples, batch_size, train_num_samples%batch_size))
        logger.warning('Try this formula: int(int({0}*{1})*{2}) % 128'.format(x.shape[0], 1.0-test_size, 1.0-validation_split))
    train_num_samples = int(x_train.shape[0] * 2 * (1.0-validation_split))
    val_num_samples = x_train.shape[0] - train_num_samples
    batch_size = 128
    if (train_num_samples%batch_size) < 100:
        logger.warning('The last batch for training after mixing could be too few! ({0}%{1})={2}. Please change test_size.'.format(train_num_samples, batch_size, train_num_samples%batch_size))
        logger.warning('Try this formula: int(int({0}*{1})*2*{2}) % 128'.format(x.shape[0], 1.0-test_size, 1.0-validation_split))
    return x_train, x_test, y_train, y_test, dxy_train, dxy_test

In [4]:
# Import muon data
# 'x' is the array of input variables, 'y' is the q/pT
x_train_displ, x_test_displ, y_train_displ, y_test_displ, dxy_train_displ, dxy_test_displ= muon_data_split(infile_muon_displ, 
                                                                                                           reg_pt_scale=reg_pt_scale, 
                                                                                                           reg_dxy_scale=reg_dxy_scale, 
                                                                                                           test_size=0.315)

y_train_displ = np.abs(y_train_displ)
y_test_displ = np.abs(y_test_displ)


[INFO    ] Loading muon data from ./data/NN_input_params_FlatXYZ.npz ...
[INFO    ] Loaded the variables with shape (19300000, 25)
[INFO    ] Loaded the parameters with shape (19300000, 6)
[INFO    ] Loaded the encoded variables with shape (3284620, 23)
[INFO    ] Loaded the encoded parameters with shape (3284620,)
[INFO    ] Loaded # of training and testing events: (2249964, 1034656)
[WARNING ] The last batch for training could be too few! (2024967%128)=7. Please change test_size.
[WARNING ] Try this formula: int(int(3284620*0.685)*0.9) % 128
[WARNING ] The last batch for training after mixing could be too few! (4049935%128)=15. Please change test_size.
[WARNING ] Try this formula: int(int(3284620*0.685)*2*0.9) % 128


In [5]:
from keras.models import Model
from keras.regularizers import L1L2
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, BatchNormalization, Dense, Activation
from nn_evaluate import huber_loss
from nn_training import train_model, lr_schedule
from keras.callbacks import LearningRateScheduler, TerminateOnNaN, ModelCheckpoint

In [6]:
def create_model(nvariables, lr=0.001, clipnorm=10., initializer = "glorot_uniform",
                nodes1=64, nodes2=32, nodes3=16, outnodes=2,
                l1_reg = 0.0, l2_reg = 0.0):
  
    regularizer = L1L2(l1=l1_reg, l2=l2_reg)
    bn_momentum = 0.9
    eps = 1e-4

    x = x_in = Input((nvariables,))
    x = BatchNormalization(epsilon=eps, momentum=bn_momentum,name="bn-input")(x)
    
    x = Dense(nodes1, 
               kernel_initializer=initializer,
               use_bias = False,
               kernel_regularizer = regularizer,
               name="hidden-dense-1")(x)
    x = BatchNormalization(epsilon = eps, momentum  = bn_momentum, name = "bn-1")(x)
    x = Activation(activation = "tanh",name="act_1")(x)
    
    if nodes2:
    
        x = Dense(nodes2, 
                   kernel_initializer=initializer,
                   use_bias = False,
                   kernel_regularizer = regularizer,
                   name="hidden-dense-2")(x)
        x = BatchNormalization(epsilon = eps, momentum  = bn_momentum, name = "bn-2")(x)
        x = Activation(activation = "tanh",name="act_2")(x)
        if nodes3:

            x = Dense(nodes3, 
                       kernel_initializer=initializer,
                       kernel_regularizer = regularizer,
                       use_bias = False,
                       name="hidden-dense-3")(x)
            x = BatchNormalization(epsilon = eps, momentum  = bn_momentum, name = "bn-3")(x)
            x = Activation(activation = "tanh", name="act_3")(x)

    x = Dense(outnodes,kernel_initializer = initializer,name="dense-output")(x)
    x = Activation("linear")(x)
    
    model = Model(inputs=x_in, outputs=x,name="baseline-model")
    
    adam = Adam(lr=lr, clipnorm=clipnorm)
    model.compile(optimizer=adam, 
                  loss=huber_loss, 
                  metrics=['acc','mse','mae'])
    model.summary()
    
    return model
    

In [7]:
assert(keras.backend.backend() == 'tensorflow')

normal_epochs = 300
normal_batch_size = 2000
l1_reg = 0.0
l2_reg = 0.0
learning_rate = 0.0063
gradient_clip_norm = 100.

lr_decay = LearningRateScheduler(lr_schedule, verbose=1)
terminate_on_nan = TerminateOnNaN()
model_training_checkpoint = ModelCheckpoint(monitor="val_loss", 
                                           verbose = 1,
                                           filepath = "checkpoints/model_ckpt_epoch_{epoch:02d}.hdf5",
                                           period = 5)
model = create_model(
                    nvariables = nvariables, 
                    lr = learning_rate, 
                    clipnorm = gradient_clip_norm, 
                    nodes1=20, 
                    nodes2=15, 
                    nodes3=10, 
                    outnodes=2,
                    l1_reg = l1_reg, 
                    l2_reg = l2_reg)

logger.info('Training model with l1_reg: {0} l2_reg: {1}'.format(l1_reg, l2_reg))

model, history = train_model(model, 
                      x_train_displ, 
                      np.column_stack((y_train_displ, dxy_train_displ)),
                      save_model=False, 
                      epochs=normal_epochs, 
                      batch_size=normal_batch_size,
                      callbacks=[lr_decay,terminate_on_nan, model_training_checkpoint], 
                      validation_split=0.1, 
                      verbose=True)

metrics = [len(history.history['loss']), history.history['loss'][-1], history.history['val_loss'][-1]]
logger.info('Epoch {0}/{0} - loss: {1} - val_loss: {2}'.format(*metrics))

2021-12-14 11:04:01.419570: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-14 11:04:01.420367: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/gpradhan/miniforge3/envs/mlp/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
[INFO    ] Training model with l1_reg: 0.0 l2_reg: 0.0
[INFO    ] Begin training ...


Metal device set to: Apple M1
Model: "baseline-model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
bn-input (BatchNormalization (None, 23)                92        
_________________________________________________________________
hidden-dense-1 (Dense)       (None, 20)                460       
_________________________________________________________________
bn-1 (BatchNormalization)    (None, 20)                80        
_________________________________________________________________
act_1 (Activation)           (None, 20)                0         
_________________________________________________________________
hidden-dense-2 (Dense)       (None, 15)                300       
_________________________________________________________________
bn-2 (BatchNormalizati

2021-12-14 11:04:01.640755: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-14 11:04:01.643887: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/300

Epoch 00001: LearningRateScheduler setting learning rate to 0.006300000008195639.


2021-12-14 11:04:01.994338: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

1013/1013 [==============================] - 15s 15ms/step - loss: 9.6231 - acc: 0.9414 - mse: 147.7374 - mae: 7.7878 - val_loss: 9.4975 - val_acc: 0.9419 - val_mse: 145.4284 - val_mae: 7.6934
Epoch 245/300

Epoch 00245: LearningRateScheduler setting learning rate to 0.0005025284481234848.
1013/1013 [==============================] - 15s 15ms/step - loss: 9.6245 - acc: 0.9412 - mse: 147.7811 - mae: 7.7889 - val_loss: 9.4967 - val_acc: 0.9418 - val_mse: 145.0746 - val_mae: 7.6928

Epoch 00245: saving model to checkpoints/model_ckpt_epoch_245.hdf5
Epoch 246/300

Epoch 00246: LearningRateScheduler setting learning rate to 0.0005025284481234848.
1013/1013 [==============================] - 15s 15ms/step - loss: 9.6291 - acc: 0.9413 - mse: 147.8552 - mae: 7.7924 - val_loss: 9.5019 - val_acc: 0.9418 - val_mse: 145.7794 - val_mae: 7.6964
Epoch 247/300

Epoch 00247: LearningRateScheduler setting learning rate to 0.0005025284481234848.
1013/1013 [==============================] - 15s 15ms/step 

1013/1013 [==============================] - 15s 15ms/step - loss: 9.6215 - acc: 0.9414 - mse: 147.7197 - mae: 7.7867 - val_loss: 9.5003 - val_acc: 0.9415 - val_mse: 145.1059 - val_mae: 7.6956
Epoch 272/300

Epoch 00272: LearningRateScheduler setting learning rate to 0.0003663432435132563.
1013/1013 [==============================] - 15s 15ms/step - loss: 9.6265 - acc: 0.9412 - mse: 147.7925 - mae: 7.7906 - val_loss: 9.4927 - val_acc: 0.9416 - val_mse: 145.1787 - val_mae: 7.6898
Epoch 273/300

Epoch 00273: LearningRateScheduler setting learning rate to 0.0003663432435132563.
1013/1013 [==============================] - 15s 15ms/step - loss: 9.6236 - acc: 0.9414 - mse: 147.7776 - mae: 7.7883 - val_loss: 9.4993 - val_acc: 0.9414 - val_mse: 145.2207 - val_mae: 7.6947
Epoch 274/300

Epoch 00274: LearningRateScheduler setting learning rate to 0.0003663432435132563.
1013/1013 [==============================] - 15s 15ms/step - loss: 9.6276 - acc: 0.9412 - mse: 147.7890 - mae: 7.7913 - val_los

1013/1013 [==============================] - 15s 15ms/step - loss: 9.6239 - acc: 0.9414 - mse: 147.7996 - mae: 7.7884 - val_loss: 9.4916 - val_acc: 0.9416 - val_mse: 145.0948 - val_mae: 7.6891
Epoch 299/300

Epoch 00299: LearningRateScheduler setting learning rate to 0.0002967380278278142.
1013/1013 [==============================] - 15s 15ms/step - loss: 9.6244 - acc: 0.9413 - mse: 147.7585 - mae: 7.7888 - val_loss: 9.5010 - val_acc: 0.9419 - val_mse: 145.1543 - val_mae: 7.6962
Epoch 300/300

Epoch 00300: LearningRateScheduler setting learning rate to 0.0002967380278278142.
1013/1013 [==============================] - 15s 15ms/step - loss: 9.6166 - acc: 0.9414 - mse: 147.5428 - mae: 7.7831 - val_loss: 9.4917 - val_acc: 0.9418 - val_mse: 145.3066 - val_mae: 7.6889


[INFO    ] Done training. Time elapsed: 1:16:55.644935 sec
[INFO    ] Epoch 300/300 - loss: 9.616619110107422 - val_loss: 9.491670608520508



Epoch 00300: saving model to checkpoints/model_ckpt_epoch_300.hdf5


In [8]:
import glob
import re
from keras.models import load_model
filenames = glob.glob("checkpoints/model_ckpt_*.hdf5")
files = sorted(filenames, key=lambda x:float(re.findall("(\d+)",x)[0]))
files[:10]

['checkpoints/model_ckpt_epoch_05.hdf5',
 'checkpoints/model_ckpt_epoch_10.hdf5',
 'checkpoints/model_ckpt_epoch_15.hdf5',
 'checkpoints/model_ckpt_epoch_20.hdf5',
 'checkpoints/model_ckpt_epoch_25.hdf5',
 'checkpoints/model_ckpt_epoch_30.hdf5',
 'checkpoints/model_ckpt_epoch_35.hdf5',
 'checkpoints/model_ckpt_epoch_40.hdf5',
 'checkpoints/model_ckpt_epoch_45.hdf5',
 'checkpoints/model_ckpt_epoch_50.hdf5']

In [9]:
# import glob
# import os

# list_of_files = glob.glob('checkpoints/model_ckpt_*.hdf5') # * means all if need specific format then *.csv
# latest_file = max(list_of_files, key=os.path.getctime)
# print(latest_file)

checkpoints/model_ckpt_epoch_300.hdf5


In [16]:
idxs = [i for i in range(5,305,5)]
maxs, mins, abs_mins, negs, pos, var = [], [], [],[], [], []
for i in range(len(files)):
    loaded_model = load_model(filepath = files[i])   
    for layer in loaded_model.layers:
        if layer.name == "dense-output":
            w = layer.get_weights()[0]
            maxs.append(np.max(w))
            mins.append(np.min(w))
            abs_mins.append(abs(np.min(w)))
            negs.append(np.sum(w < 0))
            pos.append(np.sum(w >= 0))
            var.append(np.var(w))

In [17]:
df = pd.DataFrame([maxs,mins,abs_mins,var,negs,pos])
df = df.transpose()
df.columns=["max","min","abs_min","variance","negs","pos"]
df.index = idxs
df
df.to_excel("dense-output.xlsx",index=True, index_label = "epoch")